# This notebook plots the basic results when aggregating the data, namely average encounters, bins and full traces

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.resources import INLINE

import functions_bondjango as bd
import functions_kinematic as fk
import functions_misc as fm
import functions_plotting as fp
import paths
import pandas as pd
import numpy as np
import h5py
# define the name to be used for the saved figures
save_name = 'averages'
line_width = 5

In [ ]:
from bokeh.themes.theme import Theme

theme = Theme(
    json={
    'attrs' : {
        'Figure' : {
            'background_fill_color': '#FFFFFF',
            'border_fill_color': '#FFFFFF',
            'outline_line_color': '#FFFFFF',
        },
#         'Grid': {
#             'grid_line_dash': [6, 4],
#             'grid_line_alpha': .3,
#         },
        'Text':
            {
                'text_font': 'Arial',
            },

        'Axis': {
            'major_label_text_color': 'black',
            'axis_label_text_color': 'black',
            'major_tick_line_color': 'black',
            'minor_tick_line_color': 'black',
            'axis_line_color': "black",
            'axis_line_width': 4
        }
    }
})

hv.renderer('bokeh').theme = theme

In [ ]:
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = 'result:succ, lighting:normal, rig:miniscope, analysis_type:aggFullCA'
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = data_all[0]['analysis_path']
print(data_path)

# load the data
# with pd.HDFStore(data_all[0]['analysis_path']) as h:
with pd.HDFStore(data_path) as h:

    # get a list of the existing keys
    keys = h.keys()
    # if it's only one key, just load the file
    if len(keys) == 1:
        data = h[keys[0]]
    else:   
        # allocate a dictionary for them
        data = {}
        # extract the animals present
        animal_list = [el.split('/')[1] for el in keys]
        # get the unique animals
        unique_animals = np.unique(animal_list)
        # for all the animals
        for animal in unique_animals:
            # allocate a dictionary for the 
            sub_dict = {}
            # get the unique dates for this animal
            date_list = [el.split('/')[2] for el in keys if animal in el]
            # for all the unique dates
            for date in date_list:
                # get the corresponding key
                current_key = [el for el in keys if (animal in el) and (date in el)][0]
                print(current_key)
                # load the table into the dictionary (minus the d at the beginning, added cause natural python naming)
                sub_dict[date[1:]] = h[current_key]
            # save the dictionary into the corresponding entry of the animal dictionary
            data[animal] = sub_dict
        
# print(data)



In [ ]:
# plot the data

# allocate a dict for the holomap
holo_dict = {}


# check if dictionary or dataframe
if isinstance(data, dict):
    # get the dates for this mouse
    mice = data.keys()
else:
    mice = ['All']
# for all the mice
for mouse in mice:
    
    # check if dictionary or dataframe
    if isinstance(data, dict):
        # get the dates for this mouse
        dates = data[mouse].keys()
    else:
        dates = ['All']
    # for all the dates
    for day in dates:
        # get the table
        if isinstance(data, dict):
            sub_data = data[mouse][day]
        else:
            sub_data = data
#         # pick arbitrary parameters
#         sub_data = sub_data.loc[:, ['delta_heading','mouse_heading']].copy()
            
        # get the column labels
        labels = sub_data.columns
        # get the number of rows
        x_axis = sub_data.shape[1]
        # get the array
        sub_array = sub_data.to_numpy()
        sub_array = fm.normalize_matrix(sub_array, axis=0)
        # create the labels
        y_labels = [((idx+0.5)/(len(labels))-0.5, el) for idx, el in enumerate(labels)]
        raw_image = hv.Image(sub_array, label=mouse+'_'+day)
        raw_image.opts(width=1000, height=800, invert_axes=True, invert_yaxis=True, 
                       invert_xaxis=True, cmap='Viridis', yticks=y_labels, tools=['hover'])

        holo_dict[(mouse, day)] = raw_image
# holo_image = hv.Layout(plot_list, group='images').cols(1)
holo_image = pn.panel(hv.HoloMap(holo_dict, kdims=['mouse', 'day']), center=True, widget_location='top')

# holo_image = hv.GridSpace(holo_dict, kdims=['mouse', 'day']).opts(plot_size=800)
holo_image

In [ ]:
# enc_parameters = data
# # print(data)
# target_parameter = ['mouse_cricket_distance']
# sorting_parameter = ['mouse_speed']

# trace_figs = []

# # get rid of nans

# # for the target pairs of target and sorting parameters
# for tar, sort in zip(target_parameter, sorting_parameter):
#     # sort the traces by a target parameter

#     # get the sorting vector by obtaining the desired parameter based on group by and the encounter and trial_id
#     sorting_vector = np.argsort(enc_parameters.loc[:, [sort] + ['encounter_id', 'trial_id']].groupby(
#                                 ['trial_id', 'encounter_id']).max().to_numpy(), axis=0)
#     # then extract the values as a single array to use for indexing
#     sorting_vector = np.array([el[0] for el in sorting_vector])
#     # get the actual encounters to be sorted in matrix form
#     plot_parameters = enc_parameters.loc[:, [tar] + ['encounter_id', 'trial_id']].groupby(
#         ['trial_id', 'encounter_id']).agg(list).to_numpy()
#     # turn them into an array, sorted by the sorting parameter
#     plot_parameters = np.array([el for sublist in plot_parameters for el in sublist])[sorting_vector]
# #     print(plot_parameters)
#     # plot the results
#     raw_image = hv.Image(plot_parameters, bounds=[0,0,len(y_labels),10])
#     raw_image.opts(width=1000, height=800, invert_axes=False, invert_yaxis=True, 
#                    invert_xaxis=True, cmap='Viridis', tools=['hover'])
    
#     trace_figs.append(raw_image)

# #     trace_figs[counter].savefig(join(save_path, 'trials_param' + variable_names[tar] + '_sortedby'
# #                                      + variable_names[sort] +
# #                                      '_' + out_keyword + '_' + cond_keyword + '.png'), bbox_inches='tight')

In [ ]:
# holo_image = hv.HoloMap(trace_figs.append(raw_image), kdims='Dates')
# test_panel = pn.panel(holo_image, center=True, widget_location='top')
# test_panel
# raw_image

In [ ]:
# plot an ethogram


# allocate a dict for the holomap
holo_dict = {}
# select the behavioral variables to show
target_behavior = ['cricket_0_mouse_distance','mouse_speed','cricket_0_speed','cricket_0_delta_head']
# define the plot offset
offset = 1
# define the target mouse, date and trial
mouse = 'MM_191108_a'
day = '2019_12_11'
trial = 1


# get the table
if isinstance(data, dict):
    sub_data = data[mouse][day]
else:
    sub_data = data

# get the available columns
labels = list(sub_data.columns)
# print(labels)
cells = [el for el in labels if 'cell' in el]
# calculate the average neuronal response
sub_data = sub_data.loc[sub_data['trial_id']==0, target_behavior + cells]
sub_data = sub_data.loc[5:, :]

# separate the cells and non-cells
behavior_data = sub_data[target_behavior]
sub_data = sub_data[cells]
# get the column labels
labels = sub_data.columns
# get the number of rows
x_axis = sub_data.shape[0]
# get the array
sub_array = sub_data.to_numpy().T
sub_array = fm.normalize_matrix(sub_array, axis=1)

# plot the behavioral variables
traces = []
# get the x axis values
x = np.array(np.arange(x_axis))
# get the labels of the behavioral variables
behavior_labels = behavior_data.columns

# replace the delta head name for heading
behavior_labels = ['heading' if el=='cricket_0_delta_head' else el for el in behavior_labels]
behavior_labels = [el.replace('cricket_0_', 'cricket_') if 'cricket_0_' in el else el for el in behavior_labels]

# define the ticks
y_tick = [(idx-0.5, el.replace('_',' ')) for idx, el in enumerate(behavior_labels)]
# for all the behavioral variables
for idx, variable in enumerate(target_behavior):
    
        # if it's the heading
    if variable == 'cricket_0_delta_head':
        # draw a line through the middle of the heading parameter
        trace2 = hv.Curve((x, 0.5+offset*(idx-1)), 'Time',variable)
        trace2.opts(width=fp.pix(15), height=fp.pix(6*0.2), xticks=0, xlabel='', yticks=0, ylabel='', 
               toolbar=None, color='red', line_dash='dotted', line_width=7)
        
        traces.append(trace2)
    
    y = behavior_data[variable].to_numpy()
    if variable != 'cricket_0_delta_head':
        y = np.log(y)
        y[np.isnan(y)] = 0
        y[np.isinf(y)] = 0
            
    y = fm.normalize_matrix(y, axis=0)+offset*(idx-1)
    
    trace = hv.Curve((x, y), 'Time',variable)
#     trace = hv.Curve((x, behavior_data[variable].to_numpy()+offset*(idx-1)), 'Time',variable)
    trace.opts(width=fp.pix(15), height=fp.pix(8*0.3), xticks=0, xlabel='', yticks=y_tick, ylabel='', 
               toolbar=None, color='black', line_width=7)
#     if variable == 'mouse_speed':
#         print(fm.normalize_matrix(np.log(behavior_data[variable].to_numpy()), axis=0))

    # append
    traces.append(trace)
    

    
# plot the cells
raw_image = hv.Image(sub_array, ['Time (s)','Cells'], bounds=[0,-0.5,x_axis,0.5], fontstyle='regular')

raw_image.opts(width=fp.pix(15), height=fp.pix(8*0.7), colorbar=True, colorbar_opts={'major_label_text_align': 'left'},
               cmap='Viridis', yticks=[(-0.5, 1), (0.5, sub_array.shape[0])], toolbar=None,fontsize=fp.font_sizes['small'], 
               hooks=[fp.margin])

# create the overlay with the behavioral traces
overlay = hv.Overlay(traces)
overlay.opts(text_font='Arial',fontsize=fp.font_sizes['small'])

# create the final layout
layout = hv.Layout(raw_image+overlay).cols(1)
layout.opts(toolbar=None, fontsize=fp.font_sizes['small'])

# assemble the save path
save_path = os.path.join(paths.figures_path,'_'.join((save_name,mouse,day,str(trial),'ethogram.png')))
hv.save(layout,save_path)

layout

In [ ]:
# only calculate encounter averages if encounters are present
if 'Enc' in data_path:
    encounter_angle_variables = ['mouse_heading', 'cricket_heading', 'delta_heading']
    encounter_nonangle_variables = ['mouse_cricket_distance', 'mouse_speed', 'mouse_acceleration', 'cricket_speed',
                               'cricket_acceleration']
    # allocate memory for the output
    plot_dict = {}

    # check if dictionary or dataframe
    if isinstance(data, dict):
        # get the dates for this mouse
        mice = data.keys()
    else:
        mice = ['All']
    # for all the mice
    for mouse in mice:

        # check if dictionary or dataframe
        if isinstance(data, dict):
            # get the dates for this mouse
            dates = data[mouse].keys()
        else:
            dates = ['All']
        # for all the dates
        for day in dates:
            # allocate memory for the layout
            plot_container = {}
            # get the table
            if isinstance(data, dict):
                sub_data = data[mouse][day]
            else:
                sub_data = data

            # select the data
            enc_parameters = sub_data.reset_index()

            angled_average = pd.DataFrame(
                fk.wrap(enc_parameters.loc[:, encounter_angle_variables + ['index']].groupby('index').agg(
                    lambda x: 180 + fk.circmean_deg(x))), columns=encounter_angle_variables)
            nonangled_average = enc_parameters.loc[:, encounter_nonangle_variables + ['index']].groupby('index').mean()
            encounter_average = pd.concat((angled_average, nonangled_average), axis=1)

            angled_std = pd.DataFrame(fk.unwrap(enc_parameters.loc[:, encounter_angle_variables + ['index']].groupby('index').agg(
                lambda x: fk.circstd_deg(x)/np.sqrt(x.shape[0]))), columns=encounter_angle_variables)
            nonangled_std = enc_parameters.loc[:, encounter_nonangle_variables + ['index']].groupby('index').sem()
#             print(enc_parameters.loc[:, encounter_nonangle_variables + ['frame']])
            encounter_sem = pd.concat((angled_std, nonangled_std), axis=1)

            # plot the results
            # define the variables to plot from
            encounter_variables = encounter_angle_variables + encounter_nonangle_variables
            # get the trials present
            trial_list = np.unique(enc_parameters['trial_id'])
            # get the time vector
            time_vector = enc_parameters.loc[(enc_parameters['event_id'] == 0) & 
                                             (enc_parameters['trial_id'] == trial_list[0]),
                                             'time_vector']

            # for all the variables
            for var_count, variables in enumerate(encounter_variables):
                x = time_vector.to_numpy()
                y = encounter_average.loc[:, variables].to_numpy()
                yerr = encounter_sem.loc[:, variables].to_numpy()

                plot_container[variables] = hv.Spread((list(x), list(y), list(yerr)), 
                                                      label=mouse+day).opts(title=variables)* \
                hv.Curve((list(x), list(y))).opts(color='black')
            # create the layout and store in a dict
            plot_dict[(mouse,day)] = hv.GridSpace(plot_container, kdims=['variable'])

#     encounters = hv.Layout(plot_container, group='encounters').cols(4)
    encounters = hv.GridSpace(plot_dict, kdims=['mouse', 'day']).opts(plot_size=300)
    # encounters

#     full_layout = hv.Layout(holo_image+encounters)
    full_layout = encounters
    
else:
    full_layout = holo_image

full_panel = pn.panel(full_layout, center=True, widget_location='top')
full_panel

In [ ]:
print(full_layout)


In [ ]:
try:
    full_panel.save(snakemake.output[0], resources=INLINE)
except NameError:
    print('Not snakemake')